In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, PromptTuningConfig
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
emotion_tokens = ['<happy>', '<sad>']

tokenizer = AutoTokenizer.from_pretrained('gpt2', token=HF_TOKEN)
tokenizer.add_special_tokens({"additional_special_tokens": emotion_tokens})

2

In [ ]:
train_data = [
    ('<happy> Once upon a time, there was a dragon who',
     'The dragon breathed colorful fireworks that lit up the sky.'),
    ('<sad> In a dark forest, a lonely knight',
     'The Knight knelt by the withered tree, tears falling on his rusted armor.')
]

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2", token=HF_TOKEN).to(device)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(50259, 768)

In [ ]:
peft_config = PromptTuningConfig(
    task_type="CAUSAL_LM",
    num_virtual_tokens=10,
    token_dim=model.config.hidden_size
)
print(model.config.hidden_size)

model = get_peft_model(model, peft_config)

768


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)


for epoch in range(30):
  for prompt, continuation in train_data:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    labels = tokenizer(continuation, return_tensors="pt").input_ids.to(device)

    full_inputs = torch.cat([inputs.input_ids, labels], dim=1)
    outputs = model(full_inputs, labels=full_inputs)

    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [ ]:
def generate_story(emotion, prompt):
  inputs = tokenizer(f'{emotion} {prompt}', return_tensors="pt").to(device)

  output = model.generate(
      **inputs,
      max_new_tokens=50,
      temperature=0.9,
      top_k=40,
      repetition_penalty=1.5,
      do_sample=True
  )

  return tokenizer.decode(output[0], skip_special_tokens=False)

In [ ]:
print(generate_story('<happy>', 'In a magical kingdom'))
print("=" * 100)
print(generate_story('<sad>', 'In a magical kingdom'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:1926: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<happy> In a magical kingdom for the last
 I think he, as of it's about to be. "I would you and all that she was by his (if this is not so much my will go no use in making up just one day," which way on how
<sad> In a magical kingdom. ( for an all " in the number of people, and as we have to do not even be on that it is
<|endoftext|>
